# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fba08dbbf70>
├── 'a' --> tensor([[ 0.9744, -0.0388, -2.8178],
│                   [ 0.1900,  0.7581,  0.9392]])
└── 'x' --> <FastTreeValue 0x7fba08dbbe50>
    └── 'c' --> tensor([[-1.0762, -0.0587, -0.4884,  0.7352],
                        [ 0.6031,  1.4878,  0.2406,  0.2176],
                        [-0.7667, -0.2524,  0.0169, -0.8929]])

In [4]:
t.a

tensor([[ 0.9744, -0.0388, -2.8178],
        [ 0.1900,  0.7581,  0.9392]])

In [5]:
%timeit t.a

62.5 ns ± 0.0553 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fba08dbbf70>
├── 'a' --> tensor([[ 0.3637, -0.3247, -0.1585],
│                   [ 0.7339, -0.8902,  0.8039]])
└── 'x' --> <FastTreeValue 0x7fba08dbbe50>
    └── 'c' --> tensor([[-1.0762, -0.0587, -0.4884,  0.7352],
                        [ 0.6031,  1.4878,  0.2406,  0.2176],
                        [-0.7667, -0.2524,  0.0169, -0.8929]])

In [7]:
%timeit t.a = new_value

59.4 ns ± 0.0351 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.9744, -0.0388, -2.8178],
               [ 0.1900,  0.7581,  0.9392]]),
    x: Batch(
           c: tensor([[-1.0762, -0.0587, -0.4884,  0.7352],
                      [ 0.6031,  1.4878,  0.2406,  0.2176],
                      [-0.7667, -0.2524,  0.0169, -0.8929]]),
       ),
)

In [10]:
b.a

tensor([[ 0.9744, -0.0388, -2.8178],
        [ 0.1900,  0.7581,  0.9392]])

In [11]:
%timeit b.a

57.7 ns ± 0.0241 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.9947,  0.1729,  0.8225],
               [ 1.3930, -0.4951, -1.2064]]),
    x: Batch(
           c: tensor([[-1.0762, -0.0587, -0.4884,  0.7352],
                      [ 0.6031,  1.4878,  0.2406,  0.2176],
                      [-0.7667, -0.2524,  0.0169, -0.8929]]),
       ),
)

In [13]:
%timeit b.a = new_value

495 ns ± 0.326 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

837 ns ± 0.973 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 25.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

138 µs ± 410 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

141 µs ± 431 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fba08d5ac70>
├── 'a' --> tensor([[[ 0.9744, -0.0388, -2.8178],
│                    [ 0.1900,  0.7581,  0.9392]],
│           
│                   [[ 0.9744, -0.0388, -2.8178],
│                    [ 0.1900,  0.7581,  0.9392]],
│           
│                   [[ 0.9744, -0.0388, -2.8178],
│                    [ 0.1900,  0.7581,  0.9392]],
│           
│                   [[ 0.9744, -0.0388, -2.8178],
│                    [ 0.1900,  0.7581,  0.9392]],
│           
│                   [[ 0.9744, -0.0388, -2.8178],
│                    [ 0.1900,  0.7581,  0.9392]],
│           
│                   [[ 0.9744, -0.0388, -2.8178],
│                    [ 0.1900,  0.7581,  0.9392]],
│           
│                   [[ 0.9744, -0.0388, -2.8178],
│                    [ 0.1900,  0.7581,  0.9392]],
│           
│                   [[ 0.9744, -0.0388, -2.8178],
│                    [ 0.1900,  0.7581,  0.9392]]])
└── 'x' --> <FastTreeValue 0x7fba08d5abb0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.5 µs ± 92.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fb95ba0da00>
├── 'a' --> tensor([[ 0.9744, -0.0388, -2.8178],
│                   [ 0.1900,  0.7581,  0.9392],
│                   [ 0.9744, -0.0388, -2.8178],
│                   [ 0.1900,  0.7581,  0.9392],
│                   [ 0.9744, -0.0388, -2.8178],
│                   [ 0.1900,  0.7581,  0.9392],
│                   [ 0.9744, -0.0388, -2.8178],
│                   [ 0.1900,  0.7581,  0.9392],
│                   [ 0.9744, -0.0388, -2.8178],
│                   [ 0.1900,  0.7581,  0.9392],
│                   [ 0.9744, -0.0388, -2.8178],
│                   [ 0.1900,  0.7581,  0.9392],
│                   [ 0.9744, -0.0388, -2.8178],
│                   [ 0.1900,  0.7581,  0.9392],
│                   [ 0.9744, -0.0388, -2.8178],
│                   [ 0.1900,  0.7581,  0.9392]])
└── 'x' --> <FastTreeValue 0x7fb9e4276d30>
    └── 'c' --> tensor([[-1.0762, -0.0587, -0.4884,  0.7352],
                        [ 0.6031,  1.4878,  0.2406,  0.2176],
                 

In [23]:
%timeit t_cat(trees)

30.1 µs ± 60 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59.9 µs ± 67.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-1.0762, -0.0587, -0.4884,  0.7352],
                       [ 0.6031,  1.4878,  0.2406,  0.2176],
                       [-0.7667, -0.2524,  0.0169, -0.8929]],
              
                      [[-1.0762, -0.0587, -0.4884,  0.7352],
                       [ 0.6031,  1.4878,  0.2406,  0.2176],
                       [-0.7667, -0.2524,  0.0169, -0.8929]],
              
                      [[-1.0762, -0.0587, -0.4884,  0.7352],
                       [ 0.6031,  1.4878,  0.2406,  0.2176],
                       [-0.7667, -0.2524,  0.0169, -0.8929]],
              
                      [[-1.0762, -0.0587, -0.4884,  0.7352],
                       [ 0.6031,  1.4878,  0.2406,  0.2176],
                       [-0.7667, -0.2524,  0.0169, -0.8929]],
              
                      [[-1.0762, -0.0587, -0.4884,  0.7352],
                       [ 0.6031,  1.4878,  0.2406,  0.2176],
                       [-0.7667, -0.2524,  0.0169, -0.8929]],

In [26]:
%timeit Batch.stack(batches)

79.1 µs ± 102 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-1.0762, -0.0587, -0.4884,  0.7352],
                      [ 0.6031,  1.4878,  0.2406,  0.2176],
                      [-0.7667, -0.2524,  0.0169, -0.8929],
                      [-1.0762, -0.0587, -0.4884,  0.7352],
                      [ 0.6031,  1.4878,  0.2406,  0.2176],
                      [-0.7667, -0.2524,  0.0169, -0.8929],
                      [-1.0762, -0.0587, -0.4884,  0.7352],
                      [ 0.6031,  1.4878,  0.2406,  0.2176],
                      [-0.7667, -0.2524,  0.0169, -0.8929],
                      [-1.0762, -0.0587, -0.4884,  0.7352],
                      [ 0.6031,  1.4878,  0.2406,  0.2176],
                      [-0.7667, -0.2524,  0.0169, -0.8929],
                      [-1.0762, -0.0587, -0.4884,  0.7352],
                      [ 0.6031,  1.4878,  0.2406,  0.2176],
                      [-0.7667, -0.2524,  0.0169, -0.8929],
                      [-1.0762, -0.0587, -0.4884,  0.7352],
                   

In [28]:
%timeit Batch.cat(batches)

141 µs ± 460 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

304 µs ± 496 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
